# Step 2: Viterbi algorithm

In [1]:
import matplotlib.pyplot as plt
from random import randint
from sklearn.metrics import confusion_matrix

We begin by reloading the Data_Munging file

In [2]:
%run 'Data_Munging.ipynb'

<class 'pandas.tslib.Timedelta'>
61.0


In [3]:
transitionDistribution

,0,1,4,5,10,13,15,17
0,0.990937,0.001203,0.226496,0.094737,0.001263,0.202703,0.026201,0.555556
1,0.001660,0.998231,0.004274,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.004151,0.000425,0.636752,0.000000,0.002685,0.000000,0.004367,0.000000
5,0.000899,0.000000,0.021368,0.905263,0.000000,0.000000,0.000000,0.000000
10,0.000553,0.000000,0.081197,0.000000,0.995736,0.000000,0.000000,0.000000
13,0.000553,0.000000,0.017094,0.000000,0.000316,0.797297,0.000000,0.037037
15,0.000277,0.000142,0.008547,0.000000,0.000000,0.000000,0.965066,0.000000
17,0.000969,0.000000,0.004274,0.000000,0.000000,0.000000,0.004367,0.407407


In [4]:
observationDistribution

,0,1,4,5,10,13,15,17
1,0.002905,0.000000,0.000000,0.000000,0.000000,0.067568,0.056769,0.000000
5,0.016947,0.000283,0.012821,0.878947,0.151453,0.000000,0.000000,0.000000
6,0.041779,0.000354,0.226496,0.000000,0.023215,0.000000,0.004367,0.000000
7,0.005257,0.000000,0.000000,0.000000,0.000000,0.000000,0.004367,0.111111
8,0.171751,0.000142,0.004274,0.000000,0.000000,0.297297,0.091703,0.777778
9,0.003113,0.000000,0.000000,0.000000,0.000000,0.040541,0.257642,0.000000
12,0.440963,0.999080,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000
13,0.008854,0.000000,0.000000,0.000000,0.000000,0.000000,0.017467,0.000000
14,0.084388,0.000000,0.517094,0.121053,0.109128,0.081081,0.000000,0.000000
17,0.009615,0.000142,0.000000,0.000000,0.000000,0.162162,0.100437,0.000000


On cherche la distribution initiale:

In [5]:
firstActivity = [] 
for d in range(1,32):
    dayActivity = [activityData['Start time'][x].day == d and activityData['End time'][x].day == d  for x in range(1,len(activityData))]
    if sum(dayActivity) != 0: 
        dayActivity = dayActivity + [False]
        dayActivityData = activityData[dayActivity]
        dayActivityData.index = np.arange(1,len(dayActivityData)+1)
        firstActivity.append(dayActivityData.iloc[0].ID)

In [6]:
firstActivity

[4, 4, 4, 4, 4, 4, 17, 10, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 10, 4, 4, 4, 4]

In [7]:
len(firstActivity)

24

In [8]:
firstActDict = defaultdict(int)
for act in firstActivity: 
    firstActDict[act] += 1

In [9]:
firstActDict

defaultdict(int, {4: 21, 10: 2, 17: 1})

In [10]:
initialDistribution = Series(0.0, index = [0,1,4,5,10,13,15,17])
initialDistribution

0     0
1     0
4     0
5     0
10    0
13    0
15    0
17    0
dtype: float64

In [11]:
for ind in initialDistribution.index: 
    initialDistribution[ind] = float(firstActDict[ind])/len(firstActivity)
initialDistribution

0     0.000000
1     0.000000
4     0.875000
5     0.000000
10    0.083333
13    0.000000
15    0.000000
17    0.041667
dtype: float64

In [12]:
activityLabel

0               nothing
1           leave house
4            use toilet
5           take shower
10            go to bed
13    prepare breakfast
15       prepare dinner
17            get drink
dtype: object

In [13]:
def Viterbi(observation, activityLabel, initialDistribution, observationDistribution, transitionDistribution):
    probabilityTable = pd.DataFrame(0.0, index = activityLabel.index, columns = observation.index)
    path = {}
    for activityIndex in activityLabel.index.values:
        probabilityTable[0][activityIndex] = initialDistribution[activityIndex] * observationDistribution[activityIndex][observation[0]]
        path[activityIndex] = [activityIndex]
    
    for observationIndex in range(1,len(observation)):
        newPath = {}
        for activityIndex in activityLabel.index.values:
            (proba, previousActivityInd) = max((probabilityTable[observationIndex-1][previousActivityIndex]*transitionDistribution[previousActivityIndex][activityIndex]*observationDistribution[activityIndex][observation[observationIndex]], previousActivityIndex) for previousActivityIndex in activityLabel.index.values )
            #print proba 
            probabilityTable[observationIndex][activityIndex] = proba
            newPath[activityIndex] = path[previousActivityInd] + [activityIndex]
        path = newPath
    
    (finalProba, finalActivityIndex) = max((probabilityTable[len(observation)-1][activityIndex], activityIndex) for activityIndex in activityLabel.index.values )
    return finalProba, path[finalActivityIndex]
        
        
        

Now we will create the testing set:

In [14]:
def crossValidation():
    dayList = testDay[:1]
    for d in dayList:
        print d
        daySensor = [sensorData['Start time'][x].day == d and sensorData['End time'][x].day == d for x in range(1,len(sensorData))]
        daySensor = daySensor + [False]
        daySensorData = sensorData[daySensor]
        daySensorData.index = np.arange(1,len(daySensorData)+1)
        
        dayActivity = [activityData['Start time'][x].day == d and activityData['End time'][x].day == d for x in range(1,len(activityData))]
        dayActivity = dayActivity + [False]
        dayActivityData = activityData[dayActivity]
        dayActivityData.index = np.arange(1,len(dayActivityData)+1)
        
        testingFeatureMatrix, testingLabel = convert2LastFiredFeatureMatrix(daySensorData,dayActivityData,60)
        
        testingObservation = Series(0, index = testingFeatureMatrix.columns)
        for i in range(len(testingObservation)): 
            r = np.where(testingFeatureMatrix[i] == 1)
            result = testingFeatureMatrix.index[r[0]]
            testingObservation[i] = result[0]
            
        errorRate = []
        numOfEvents = []
        #for x in range(1,len(testingObservation),150): 
        obs = testingObservation[:len(testingObservation)]
        validatedPath = testingLabel[:len(testingObservation)]
        proba, path = Viterbi(obs, activityLabel, initialDistribution, observationDistribution, transitionDistribution)
        errorRate.append(  (sum(validatedPath != path)/float(len(validatedPath))) * 100 )
        numOfEvents.append(len(testingObservation))
        validatedPath2 = pd.Series(validatedPath, name = "Actual") 
        path2 = pd.Series(path, name = "Predicted")
        return pd.crosstab(validatedPath2, path2), pd.crosstab(validatedPath2, path2).apply(lambda r: r/r.sum(), axis=0)
        #plt.plot(numOfEvents,errorRate, label = d)
         
    #plt.xlabel("Number of observations in the training sequence")
    #plt.ylabel("Error rate (%)")
    #plt.legend(loc='upper right')
    #plt.show()

In [15]:
confusionMatrix, confusionMatrixNormalized = crossValidation()

25


In [16]:
activityLabel

0               nothing
1           leave house
4            use toilet
5           take shower
10            go to bed
13    prepare breakfast
15       prepare dinner
17            get drink
dtype: object

In [17]:
confusionMatrix

Predicted,0,1,4,5,10,13,15,17
Actual,,,,,,,,
0,12,1,73,43,18,5,35,183
1,1,397,0,0,0,0,0,0
4,0,0,6,0,0,0,0,4
5,0,0,0,11,0,0,0,0
10,0,0,0,0,552,0,0,0
13,0,0,0,0,0,5,0,0
15,1,0,0,0,0,12,27,2
17,0,0,0,0,0,0,0,4


In [18]:
def plotConfusionMatrix(confusionMatrix, title='Confusion matrix', cmap=plt.cm.gray_r):
    plt.matshow(confusionMatrix, cmap=cmap) # imshow
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(confusionMatrix.columns))
    plt.xticks(tick_marks, confusionMatrix.columns, rotation=45)
    plt.yticks(tick_marks, confusionMatrix.index)
    plt.ylabel(confusionMatrix.index.name)
    plt.xlabel(confusionMatrix.columns.name)
    plt.show()

In [19]:
plotConfusionMatrix(confusionMatrixNormalized)

//anaconda/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [20]:
activityLabel

0               nothing
1           leave house
4            use toilet
5           take shower
10            go to bed
13    prepare breakfast
15       prepare dinner
17            get drink
dtype: object

In [21]:
testDay[2:3]

[5]

In [22]:
sensorLabel

1              Microwave
5       Hall-Toilet door
6     Hall-Bathroom door
7          Cups cupboard
8                 Fridge
9        Plates cupboard
12            Front door
13            Dishwasher
14           ToiletFlush
17               Freezer
18         Pans Cupboard
20       Washing machine
23    Groceries Cupboard
24     Hall-Bedroom door
dtype: object